# Output best placements of new sensors
Load in our virtual sensor pollution prediction model, load in our features for the past 7 days, make predictions.  
Create a dataframe with the most polluted locations to the least polluted.

In [ ]:
import pandas as pd
import datetime
import numpy as np
from copy import deepcopy
from collections import defaultdict
from joblib import load

# angshuman's functions
from HistoricalData.getData import getNearestEpaData, getNearestNoaaData, get_data

#### Load features

In [ ]:
X_data_df = pd.read_csv("xfile_rf.csv") # or whatever the filename is saved from Jake's build on the static csv

In [ ]:
# mean interpolating where we don't have humidity/temperature
# ideally we do this kind of final cleanup in the previous step where we create the csv of features
for col in ['temperature', 'humidity']:
    v = X_data_df[col].mean()
    X_data_df[col].fillna(v, inplace = True)
    X_data_df[col].fillna(v, inplace = True)

#### loading model from file

In [ ]:
model = load("nn/201901103_RF.joblib") # or whatever the filename is of the latest model save

#### make predictions

In [ ]:
preds = model.predict(X_data_df.to_numpy())

#### create dataframe for map to query

In [ ]:
map_df = pd.DataFrame(X_data_df[['lat','lon', 'created']])
map_df['coords'] = list(zip(map_df.lat, map_df.lon))
map_df['pred'] = preds

#### average predictions at each sensor over the time window

In [ ]:
avg_preds = map_df.groupby('coords')['pred'].mean()
map_df = map_df.drop_duplicates(subset='coords')
map_df['avg_preds'] = avg_preds
map_df.sort('avg_preds', ascending=False, inplace = True)

In [ ]:
# should be able to take this dataframe, drop records outside lat lon range, and grab the top x number of values

## Old code for plotting on a map

In [ ]:
f, a = plt.subplots(figsize = (10,10))
a.scatter(x = df_tmp['x'],
          y = df_tmp['y'],
          c = preds,
          cmap = 'coolwarm',
          s = 10)

In [ ]:
plt.hist(preds)

I mean,that's kind of what our distribution looks like

### what if we log-transform the output

In [ ]:
plt.hist(np.log(preds))

In [ ]:
f, a = plt.subplots(figsize = (10,10))
a.scatter(x = df_tmp['x'],
          y = df_tmp['y'],
          c = np.log(preds),
          cmap = 'seismic',
          s = 10)

**that looks better**

# important note, read me
Note the dark red near like, land's end. It's possible there is someone smoking there near a sensor and it's making all the neighbors guess slightly more red than usual. We should drop PA sensor readings over a certain threshold (say, 300) so an outlier reading doesn't throw off all neighbor readings